# About

The script reads the Electricity Demand components from the ETYS Spatial CSV file created by running SAS script "Write_ETYS-Demands-CSV.sas". It processes it to produce the CSV files required for the visualisation map.

In [34]:
#Get the list of GSPs in the spatial visualisation
import json
with open("/home/rob/workNG/regional-fes-resources/Geographies/GSPs 2019/GSP_post.geojson") as f:
    data = json.load(f) 

fin = open("./GSPs_VisualisationList.csv", "wt")
fin.write("GSP(s)")
fin.write('\n')

for feature in data['features']:
    fin.write(feature['properties']['GSP(s)'])
    fin.write('\n')
    
fin.close()

In [1]:
import pandas as pd

# Read in the CSV to "df_csv"
df_csv = pd.read_csv("/home/rob/workNG/regional-fes-resources/ETYS data/Input/active.csv")
print(df_csv.head(10))

  scenario    GSP  DemandPk  DemandAM  DemandPM type  year
0       SP  ABHA1    70.149    27.455    47.595    C    19
1       SP  ABHA1    70.912    27.754    48.113    C    20
2       SP  ABHA1    71.497    27.983    48.510    C    21
3       SP  ABHA1    71.974    28.170    48.834    C    22
4       SP  ABHA1    72.314    28.302    49.064    C    23
5       SP  ABHA1    72.688    28.449    49.317    C    24
6       SP  ABHA1    72.970    28.559    49.509    C    25
7       SP  ABHA1    73.172    28.638    49.646    C    26
8       SP  ABHA1    73.399    28.727    49.800    C    27
9       SP  ABHA1    73.573    28.795    49.918    C    28


In [6]:
import shutil, os

# We delete the outputs folder and all it's content (to remove any old data).
dir_path = "/home/rob/workNG/regional-fes-resources/ETYS data/Output/"
try:
    shutil.rmtree(dir_path)
except OSError as e:
    print("Error: %s : %s" % (dir_path, e.strerror))

# We create a new empty Output folder.
try:
    os.mkdir(dir_path)
except OSError:
    print ("Creation of the directory %s failed" % dir_path)
else:
    print ("Successfully created the directory %s " % dir_path)

Successfully created the directory /home/rob/workNG/regional-fes-resources/ETYS data/Output/ 


In [7]:
for category in ["C", "D", "E", "H", "I", "R", "Z"]:
    for scenario in ["SP", "CT", "ST", "LW"]:
        # Create df as a filter of df_csv
        df = df_csv[(df_csv['scenario'] == scenario) & (df_csv['type'] == category)].drop(columns=['DemandAM', 'DemandPM'])
        df['year'] = df['year'] + 2000
        
        # Pivot to have years across the top
        df = df.pivot(index='GSP', columns='year', values='DemandPk')
        
        # We don't currently have geographic regions for all GSPs. For now let's just sum up certain rows.
        df.loc['ABHA1;ABNE_P'] = df.loc['ABHA1'] + df.loc['ABNE_P']
        df.drop(['ABHA1', 'ABNE_P'], inplace=True)
        df.loc['CANTN1;RICH1;RICH_J'] = df.loc['CANTN1'] + df.loc['RICH1'] + df.loc['RICH_J']
        df.drop(['CANTN1', 'RICH1', 'RICH_J'], inplace=True) #, errors='ignore')      
        
        # Export to CSV
        filename = scenario + "-DemandPk-" + category + ".csv"
        df.to_csv("/home/rob/workNG/regional-fes-resources/ETYS data/Output/" + filename, index=True, float_format='%.3f')

KeyError: 'RICH1'